# Turning ocean model output into “analysis ready data” (ARD)

## Ocean Hack Week 2024 - Australia 

Date: 26 August, 2024

Author = {"name": "Thomas Moore", "affiliation": "CSIRO", "email": "thomas.moore@csiro.au", "orcid": "0000-0003-3930-1946"}

## Goal: calculate SST climatology from a CMIP6 ocean model. **AND do it easily and quickly.**

#### Setting up your ARE session - dialing up your `JobFS` resources

##### Bookmark this information on NCI Queue limits
https://opus.nci.org.au/pages/viewpage.action?pageId=236881198

# Setup your Dask cluster

NB: looks like a work-around in netcdf4-python to deal with netcdf-c not being thread safe was removed in 1.6.1. The solution (for now) is to make sure your cluster only uses 1 thread per worker

Discussion:
https://forum.access-hive.org.au/t/netcdf-not-a-valid-id-errors/389

![open source dependency](https://www.explainxkcd.com/wiki/images/d/d7/dependency.png)

In [1]:
### setup dask cluster
from dask.distributed import Client
client = Client(threads_per_worker=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 28
Total threads: 28,Total memory: 251.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42797,Workers: 28
Dashboard: /proxy/8787/status,Total threads: 28
Started: Just now,Total memory: 251.19 GiB
Comm: tcp://127.0.0.1:44399,Total threads: 1
Dashboard: /proxy/35721/status,Memory: 8.97 GiB
Nanny: tcp://127.0.0.1:43807,


# use CMIP6 data catalogs

The global CMIP6 (Coupled Model Intercomparison Project Phase 6) dataset is enormous, reflecting the extensive range of climate simulations and variables it contains. As of recent estimates, the CMIP6 archive is expected to surpass **20 petabytes (PB)** in total size. This includes data from various modeling centers around the world, covering different experiments, variables, and temporal resolutions.

The total number of NetCDF files in the CMIP6 dataset is immense, with estimates suggesting there are **over 2 million NetCDF files**.

### Information on climate data catalogs across Australian HPC

**ACCESS-NRI** https://access-nri-intake-catalog.readthedocs.io/en/latest/usage/how.html <br>
**NCI** https://opus.nci.org.au/pages/viewpage.action?pageId=213713098


In [2]:
import intake
import xarray as xr

### Earth System Grid Federation (ESGF) Australian CMIP6-era Datasets
https://geonetwork.nci.org.au/geonetwork/srv/eng/catalog.search#/metadata/f3154_9976_7262_7595

In [3]:
cmip6_fs38_catalog = intake.open_esm_datastore("/g/data/dk92/catalog/v2/esm/cmip6-fs38/catalog.json")

In [4]:
cmip6_fs38_catalog

,unique
path,1054133
file_type,2
realm,7
frequency,10
table_id,24
project_id,1
institution_id,3
source_id,4
experiment_id,52
member_id,80


In [5]:
cmip6_fs38_catalog.df

,path,file_type,realm,frequency,table_id,project_id,institution_id,source_id,experiment_id,member_id,variable_id,grid_label,version,time_range
0,/g/data/fs38/publications/CMIP6/ScenarioMIP/CS...,f,ocnBgchem,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,ssp370,r3i1p1f1,fgco2nat,gn,d20191203,201501-210012
1,/g/data/fs38/publications/CMIP6/ScenarioMIP/CS...,f,land,day,Eday,CMIP,CSIRO,ACCESS-ESM1-5,ssp370,r3i1p1f1,mrsll,gn,d20191203,20200101-20241231
2,/g/data/fs38/publications/CMIP6/ScenarioMIP/CS...,f,land,day,Eday,CMIP,CSIRO,ACCESS-ESM1-5,ssp370,r3i1p1f1,lai,gn,d20191203,20650101-21001231
3,/g/data/fs38/publications/CMIP6/ScenarioMIP/CS...,f,atmos,day,day,CMIP,CSIRO,ACCESS-ESM1-5,ssp370,r3i1p1f1,hursmax,gn,d20191203,20650101-21001231
4,/g/data/fs38/publications/CMIP6/ScenarioMIP/CS...,l,atmos,mon,Amon,CMIP,CSIRO,ACCESS-ESM1-5,ssp370,r20i1p1f1,rlus,gn,v20210525,201501-210012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1054128,/g/data/fs38/publications/CMIP6/DAMIP/CSIRO/AC...,f,atmos,day,day,CMIP,CSIRO,ACCESS-ESM1-5,ssp245-cov-modgreen,r2i1p1f1,ua,gn,d20210316,20500101-20501231
1054129,/g/data/fs38/publications/CMIP6/DAMIP/CSIRO/AC...,l,atmos,mon,Amon,CMIP,CSIRO,ACCESS-ESM1-5,ssp245-cov-modgreen,r2i1p1f1,tauu,gn,v20210316,202001-205012
1054130,/g/data/fs38/publications/CMIP6/DAMIP/CSIRO/AC...,f,ocean,mon,Omon,CMIP,CSIRO,ACCESS-ESM1-5,ssp245-cov-modgreen,r2i1p1f1,hfsso,gn,d20210316,202001-205012
1054131,/g/data/fs38/publications/CMIP6/DAMIP/CSIRO/AC...,l,land,mon,Lmon,CMIP,CSIRO,ACCESS-ESM1-5,ssp245-cov-modgreen,r2i1p1f1,treeFrac,gn,v20210316,202001-205012


### Variable name search here? https://pcmdi.llnl.gov/mips/cmip3/variableList.html

In [6]:
my_cmip6_search = cmip6_fs38_catalog.search(
experiment_id="historical",
realm='ocean',
source_id='ACCESS-CM2',
variable_id='tos',
frequency='mon',
file_type='f'
)
my_cmip6_search

,unique
path,15
file_type,1
realm,1
frequency,1
table_id,1
project_id,1
institution_id,1
source_id,1
experiment_id,1
member_id,10


In [7]:
my_cmip6_search.unique().path

['/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r5i1p1f1/Omon/tos/gn/files/d20210607/tos_Omon_ACCESS-CM2_historical_r5i1p1f1_gn_185001-201412.nc',
 '/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r4i1p1f1/Omon/tos/gn/files/d20210607/tos_Omon_ACCESS-CM2_historical_r4i1p1f1_gn_185001-201412.nc',
 '/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r6i1p1f1/Omon/tos/gn/files/d20220819/tos_Omon_ACCESS-CM2_historical_r6i1p1f1_gn_185001-194912.nc',
 '/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r10i1p1f1/Omon/tos/gn/files/d20220819/tos_Omon_ACCESS-CM2_historical_r10i1p1f1_gn_195001-201412.nc',
 '/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r6i1p1f1/Omon/tos/gn/files/d20220819/tos_Omon_ACCESS-CM2_historical_r6i1p1f1_gn_195001-201412.nc',
 '/g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r9i1p1f1/Omon/tos/gn/files/d20220819/tos_Omon_ACCESS-CM2_hi

In [8]:
%%time
dataset_dict = my_cmip6_search.to_dataset_dict(progressbar=False)
combined_dataset = xr.concat(
    dataset_dict.values(),
    dim='member'
)

ValueError: {'vertices_longitude', 'vertices_latitude', 'time_bnds'} is a coordinate in some datasets but not others.

In [9]:
%%time
dataset_dict = my_cmip6_search.to_dataset_dict(progressbar=False)
dataset_dict_clean = {key: ds.drop_vars(['vertices_longitude', 'vertices_latitude', 'time_bnds'], errors='ignore') 
           for key, ds in dataset_dict.items()}
combined_ds = xr.concat(dataset_dict_clean.values(), dim='member')

CPU times: user 1.42 s, sys: 247 ms, total: 1.67 s
Wall time: 2.35 s


In [10]:
combined_ds

<xarray.Dataset> Size: 9GB
Dimensions:    (member: 10, time: 1980, j: 300, i: 360)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    longitude  (j, i) float64 864kB 80.5 81.5 82.5 83.5 ... 79.96 79.97 79.99
Dimensions without coordinates: member
Data variables:
    tos        (member, time, j, i) float32 9GB dask.array<chunksize=(1, 1, 300, 360), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            54786.0
    creation_date:                    2021-06-07T09:41:24Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         d20210607
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           f.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historic...

## what is the default NetCDF chunking?
`ncdump -hs /g/data/fs38/publications/CMIP6/CMIP/CSIRO-ARCCSS/ACCESS-CM2/historical/r5i1p1f1/Omon/tos/gn/files/d20210607/tos_Omon_ACCESS-CM2_historical_r5i1p1f1_gn_185001-201412.nc`

**tos:_ChunkSizes = 1, 300, 360 ;**

In [11]:
%%time
xarray_open_kwargs = {'chunks':{'time':495}}
dataset_dict = my_cmip6_search.to_dataset_dict(progressbar=False,xarray_open_kwargs=xarray_open_kwargs)
dataset_dict_clean = {key: ds.drop_vars(['vertices_longitude', 'vertices_latitude', 'time_bnds'], errors='ignore') 
           for key, ds in dataset_dict.items()}
combined_ds_chunked = xr.concat(dataset_dict_clean.values(), dim='member')

CPU times: user 1.3 s, sys: 272 ms, total: 1.57 s
Wall time: 2.28 s


In [12]:
combined_ds_chunked

<xarray.Dataset> Size: 9GB
Dimensions:    (member: 10, time: 1980, j: 300, i: 360)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    longitude  (j, i) float64 864kB 80.5 81.5 82.5 83.5 ... 79.96 79.97 79.99
Dimensions without coordinates: member
Data variables:
    tos        (member, time, j, i) float32 9GB dask.array<chunksize=(1, 495, 300, 360), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            54786.0
    creation_date:                    2021-06-07T09:41:24Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         d20210607
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           f.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historic...

In [13]:
SST_climatology = combined_ds.groupby('time.month').mean('time')
SST_climatology

<xarray.Dataset> Size: 54MB
Dimensions:    (month: 12, member: 10, j: 300, i: 360)
Coordinates:
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    longitude  (j, i) float64 864kB 80.5 81.5 82.5 83.5 ... 79.96 79.97 79.99
  * month      (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: member
Data variables:
    tos        (month, member, j, i) float32 52MB dask.array<chunksize=(1, 1, 300, 360), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            54786.0
    creation_date:                    2021-06-07T09:41:24Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         d20210607
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           f.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historic...

### Wall time: 3min 25s

In [14]:
%%time
SST_climatology = SST_climatology.compute()

CPU times: user 1min 35s, sys: 7.75 s, total: 1min 43s
Wall time: 3min 16s


In [15]:
del(SST_climatology)

In [16]:
SST_climatology = combined_ds_chunked.groupby('time.month').mean('time')
SST_climatology

<xarray.Dataset> Size: 54MB
Dimensions:    (month: 12, member: 10, j: 300, i: 360)
Coordinates:
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    longitude  (j, i) float64 864kB 80.5 81.5 82.5 83.5 ... 79.96 79.97 79.99
  * month      (month) int64 96B 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: member
Data variables:
    tos        (month, member, j, i) float32 52MB dask.array<chunksize=(12, 1, 300, 360), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            54786.0
    creation_date:                    2021-06-07T09:41:24Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         d20210607
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           f.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historic...

### Wall time: 32.3 s

In [17]:
%%time
SST_climatology = SST_climatology.compute()

CPU times: user 5.12 s, sys: 1.26 s, total: 6.37 s
Wall time: 32.2 s


# Rechunking for ensemble calculations

In [18]:
ensemble_mean = combined_ds_chunked.mean('member')

In [19]:
ensemble_mean

<xarray.Dataset> Size: 857MB
Dimensions:    (time: 1980, j: 300, i: 360)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    longitude  (j, i) float64 864kB 80.5 81.5 82.5 83.5 ... 79.96 79.97 79.99
Data variables:
    tos        (time, j, i) float32 855MB dask.array<chunksize=(495, 300, 360), meta=np.ndarray>

In [20]:
%%time
ensemble_mean = ensemble_mean.compute()

2024-08-25 19:28:59,557 - distributed.worker.memory - WARNING - Worker is at 81% memory usage. Pausing worker.  Process memory: 3.67 GiB -- Worker memory limit: 4.50 GiB
2024-08-25 19:28:59,558 - distributed.worker.memory - WARNING - Worker is at 65% memory usage. Resuming worker. Process memory: 2.93 GiB -- Worker memory limit: 4.50 GiB
2024-08-25 19:29:22,322 - distributed.worker - ERROR - Worker stream died during communication: tcp://127.0.0.1:41373
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker.py", line 2059, in gather_dep
   

CPU times: user 6.55 s, sys: 3.26 s, total: 9.82 s
Wall time: 55.4 s


In [21]:
ensemble_mean

<xarray.Dataset> Size: 857MB
Dimensions:    (time: 1980, j: 300, i: 360)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    longitude  (j, i) float64 864kB 80.5 81.5 82.5 83.5 ... 79.96 79.97 79.99
Data variables:
    tos        (time, j, i) float32 855MB nan nan nan nan ... nan nan nan nan

In [22]:
combined_ds_chunked

<xarray.Dataset> Size: 9GB
Dimensions:    (member: 10, time: 1980, j: 300, i: 360)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB -77.88 -77.88 -77.88 ... 66.04 65.63 65.21
    longitude  (j, i) float64 864kB 80.5 81.5 82.5 83.5 ... 79.96 79.97 79.99
Dimensions without coordinates: member
Data variables:
    tos        (member, time, j, i) float32 9GB dask.array<chunksize=(1, 495, 300, 360), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            54786.0
    creation_date:                    2021-06-07T09:41:24Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         d20210607
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           f.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historic...

In [24]:
dataset_rechunked = combined_ds_chunked.chunk({'member':10,'i':36})
dataset_rechunked

<xarray.Dataset> Size: 9GB
Dimensions:    (member: 10, time: 1980, j: 300, i: 360)
Coordinates:
  * time       (time) datetime64[ns] 16kB 1850-01-16T12:00:00 ... 2014-12-16T...
  * j          (j) int32 1kB 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * i          (i) int32 1kB 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    latitude   (j, i) float64 864kB dask.array<chunksize=(300, 36), meta=np.ndarray>
    longitude  (j, i) float64 864kB dask.array<chunksize=(300, 36), meta=np.ndarray>
Dimensions without coordinates: member
Data variables:
    tos        (member, time, j, i) float32 9GB dask.array<chunksize=(10, 495, 300, 36), meta=np.ndarray>
Attributes: (12/64)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            54786.0
    creation_date:                    2021-06-07T09:41:24Z
    ...                               ...
    intake_esm_attrs:variable_id:     tos
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         d20210607
    intake_esm_attrs:time_range:      185001-201412
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           f.CMIP.CSIRO-ARCCSS.ACCESS-CM2.historic...

In [26]:
%%time
dataset_rechunked.to_netcdf('/scratch/nf33/moore_tutorial/SST_rechunked.nc')

HDF5-DIAG: Error detected in HDF5 (1.14.3) thread 0:
  #000: H5F.c line 836 in H5Fopen(): unable to synchronously open file
    major: File accessibility
    minor: Unable to open file
  #001: H5F.c line 796 in H5F__open_api_common(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #002: H5VLcallback.c line 3863 in H5VL_file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #003: H5VLcallback.c line 3675 in H5VL__file_open(): open failed
    major: Virtual Object Layer
    minor: Can't open object
  #004: H5VLnative_file.c line 128 in H5VL__native_file_open(): unable to open file
    major: File accessibility
    minor: Unable to open file
  #005: H5Fint.c line 1910 in H5F_open(): unable to lock the file
    major: File accessibility
    minor: Unable to lock file
  #006: H5FD.c line 2412 in H5FD_lock(): driver lock request failed
    major: Virtual File Layer
    minor: Unable to lock file
  #007: H5FDsec2.c line 941 

OSError: [Errno -101] NetCDF: HDF error: '/scratch/nf33/moore_tutorial/SST_rechunked.nc'

In [ ]:
client.shutdown()

# $The$ $End$

## Plot current vectors for August

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
from matplotlib import mlab, cm, gridspec
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
%matplotlib inline 

In [ ]:
%%time
# Define the u and v components of the currents
time_choice = 8
u = clim_uv.u.sel(month=time_choice)
v = clim_uv.v.sel(month=time_choice)
speed = np.sqrt(u**2 + v**2)

In [ ]:
#plot model data
transform = ccrs.PlateCarree()
cmap = 'Spectral_r'
cbar_label='current speed'
plot_data = speed

###
fig = plt.figure(num=None, figsize=(8, 6), dpi=300, facecolor='w', edgecolor='k')
ax = plt.subplot(projection=ccrs.PlateCarree(180))
ax.set_extent([142,160, -25, -10], ccrs.PlateCarree())
ax.add_feature(cfeature.NaturalEarthFeature('physical', 'land', '50m', edgecolor='face', facecolor='white'))
ax.coastlines('50m',linewidth=0.5,edgecolor='grey')
plot_data.plot(transform=transform,cmap=cmap,cbar_kwargs={'label': cbar_label,'shrink':0.5},robust=True)

#plot u/v vectors
# Define the x and y coordinates
x = clim_uv.xu_ocean
y = clim_uv.yu_ocean
ax.quiver(x.values,y.values,u.values,v.values,transform=transform, units='x', width=0.01, scale=0.7, headwidth=2,alpha=0.2)
ax.set_title('BRAN2020 1993-2022\ncurrent speed \n August Climatology')